In [6]:
# This file is part of ts_standardscripts
#
# Developed for the LSST Telescope and Site Systems.
# This product includes software developed by the LSST Project
# (https://www.lsst.org).
# See the COPYRIGHT file at the top-level directory of this distribution
# for details of code ownership.
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License

__all__ = ["StressLOVE"]

import asyncio
import aiohttp
import json
import requests
import yaml
from astropy.time import Time
from lsst.ts import salobj


class ClientInstance:
    def __init__(self, location, username, password, event_streams, telemetry_streams):
        self.location = location
        self.username = username
        self.password = password

        self.event_streams = event_streams
        self.telemetry_streams = telemetry_streams
        
        self.websocket_url = ''
        self.received_messages = 0
        self.msg_traces = []
    
    def request_token(self):
        url = f'http://{self.location}/manager/api/get-token/'
        data = {
            'username': self.username,
            'password': self.password,
        }
        resp = requests.post(url, data = data)
        try:
            token = resp.json()['token']
        except Exception:
            raise Exception("Authentication failed")
        self.websocket_url = f'ws://{self.location}/manager/ws/subscription?token={token}'
    
    async def handle_message_reception(self):
        """Handles the reception of messages"""
        if self.websocket:
            async for message in self.websocket:
                if message.type == aiohttp.WSMsgType.TEXT:
                    msg = json.loads(message.data)
                    if 'category' not in msg or ("option" in msg and msg["option"] == "subscribe"):
                        continue
                    self.received_messages = self.received_messages + 1
                    tracing = msg["tracing"]
                    tracing['client_rcv'] = Time.now().tai.datetime.timestamp()
                    self.msg_traces.append(tracing)
    
    async def subscribe_to(self, csc, salindex, stream, topic_type):
        subscribe_msg = {
            'option': 'subscribe',
            'category': topic_type,
            'csc': csc,
            'salindex': salindex,
            'stream': stream
        }
        await self.websocket.send_str(json.dumps(subscribe_msg))
    
    async def start_ws_client(self):
        async with aiohttp.ClientSession() as session:
            self.websocket = await session.ws_connect(self.websocket_url)
            for csc, stream in self.event_streams:
                await self.subscribe_to(csc.split(':')[0], csc.split(':')[1], stream, 'event')
            for csc, stream in self.telemetry_streams:
                await self.subscribe_to(csc.split(':')[0], csc.split(':')[1], stream, 'telemetry')
                
            await self.handle_message_reception()



class StressLOVE(salobj.BaseScript):
    """Run a stress test for one or more CSCs.

    Notes
    -----
    **Checkpoints**

    * "set {csc_name}:{index}" before commanding a CSC.

    **Details**

    * Takes the shortest path from the current state to the requested state.
      Thus if you want to configure a CSC you should specify it twice:

      * First with state "STANDBY".
      * Next with state "DISABLED" or "ENABLED" and the desired settings.

    * Dynamically loads IDL files as needed.
    """

    def __init__(self, index):
        super().__init__(index=index, descr="Run a stress test for one or more CSCs")

        # number of clients to simulate on the stress test
        self.number_of_clients = 50

    @classmethod
    def get_schema(cls):
        schema_yaml = """
            $schema: http://json-schema.org/draft-07/schema#
            $id: https://github.com/lsst-ts/ts_standardscripts/StressLOVE.yaml
            title: StressLOVE v1
            description: Configuration for StressLOVE
            type: object
            properties:
              location:
                description: Domain of the running LOVE instance to stress
                type: string
              username:
                description: Username to authenticate connections
                type: string
              password:
                description: Password to authenticate User
                type: string
              data:
                description: List of CSC_name[:index]
                type: array
                minItems: 1
                items:
                    type: string
            required: [location, user, password, data]
            additionalProperties: false
        """
        return yaml.safe_load(schema_yaml)

    async def configure(self, config):
        """Configure the script.

        Specify the Stress test configurations:
        - LOVE domain location
        - User
        - Password
        - CSCs

        Parameters
        ----------
        config : `types.SimpleNamespace`
            Configuration with several attributes:
            * location : a string, with the location of the LOVE instance to stress
            * username : a string, with the username to authenticate connections
            * password : a string, with the password of the user

            * data : a list, where each element is a string in the form:

                * CSC name and optional index as ``csc_name:index`` (a `str`).
                  For a CSC that is not indexed you may omit ``:index``
                  or specify ``:0``, as you prefer.

        Notes
        -----
        Saves the results on several attributes:
        * location : a string, with the location of the LOVE instance to stress
        * username : a string, with the username to authenticate connections
        * password : a string, with the password of the user
        * remotes : a dict of (csc_name, index): remote, an `lsst.ts.salobj.Remote`

        Constructing a `salobj.Remote` is slow (DM-17904), so configuration
        may take a 10s or 100s of seconds per CSC.
        """
        self.log.info("Configure started")
        
        # set configurations
        self.location = config.location
        self.username = config.username
        self.password = config.password

        # construct remotes
        remotes = dict()
        for elt in config.data:
            name, index = salobj.name_to_name_index(elt)
            self.log.debug(f"Create remote {name}:{index}")
            if elt not in remotes:
                remote = salobj.Remote(
                    domain=self.domain, name=name, index=index
                )
                remotes[elt] = remote

        self.remotes = remotes

    def set_metadata(self, metadata):
        """Compute estimated duration.

        Parameters
        ----------
        metadata : SAPY_Script.Script_logevent_metadataC
        """
        # a crude estimate; state transitions are typically quick
        # but we don't know how many of them there will be
        metadata.duration = len(self.nameind_state_settings) * 2


    async def run(self):
        """Run script."""
        tasks = [
            remote.start_task
            for remote in self.remotes.values()
            if not remote.start_task.done()
        ]
        if tasks:
            self.log.info(f"Waiting for {len(tasks)} remotes to be ready")
            await asyncio.gather(*tasks)

        # for name_index, state, settings in self.nameind_state_settings:
        #     name, index = name_index
        #     await self.checkpoint(f"set {name}:{index}")
        #     remote = self.remotes[(name, index)]
        #     await salobj.set_summary_state(
        #         remote, state, settings, timeout=self.cmd_timeout
        #     )

        event_streams = dict()
        telemetry_streams = dict()
        for producer in self.remotes.keys():
            event_streams[producer] = salobj.SalInfo(self.domain, producer.split(':')[0]).__getattribute__(f"event_names")
            telemetry_streams[producer] = salobj.SalInfo(self.domain, producer.split(':')[0]).__getattribute__(f"telemetry_names")

        # Create clients and listen to ws messages
        loop = asyncio.get_event_loop()
        clients = [ClientInstance(
            self.location, self.username, self.password,
            event_streams, telemetry_streams,
        ) for i in range(self.number_of_clients)]

        for c in clients:
            c.request_token()
            
        print([c.websocket_url for c in clients])

        #client_tasks = [loop.create_task(c.start_ws_client()) for c in clients]


In [7]:
import types
MANAGER_LOCATION = "dev.love.inria.cl"
MANAGER_USER = "admin"
MANAGER_PASSWORD = "test"
m_config = types.SimpleNamespace(location=MANAGER_LOCATION, username=MANAGER_USER, password=MANAGER_PASSWORD, data=["ATDome:0", "ATMCS:0"])
print(m_config)

namespace(data=['ATDome:0', 'ATMCS:0'], location='dev.love.inria.cl', password='test', username='admin')


In [8]:
stress_instance = StressLOVE(1001)

In [9]:
await stress_instance.configure(m_config)

In [10]:
await stress_instance.run()

Exception: Authentication failed